In [7]:
GYRO = False
SEQUENCE_LENGTH = 4
SEQUENCE_OVERLAP = 3
BATCH_SIZE = 10
EPOCHS = 20
MODEL_NAME = f"epochs:{EPOCHS}_batch:{BATCH_SIZE}_gyro:{GYRO}_window:{SEQUENCE_LENGTH}_overlap:{SEQUENCE_OVERLAP}"
DEV_SIZE = 5
SAVE_MODEL = False
TEST_SIZE = 5
NORMALISE = False


### This code takes in the type of activity, ie "stationary" or "moving" and trains a model for just classifying which phsyical activity of that category the data is  

In [8]:
import file_tagger
import sequence_genrator
import tensorflow as tf
import numpy as np
from keras import layers, Sequential, models
import split_by_student

ALL_ACTIVITIES = [
    "sitting",
    "standing",
    "lying_down_back",
    "lying_down_stomach",
    "lying_down_right",
    "lying_down_left",
    "walking",
    "ascending_stairs",
    "descending_stairs",
    "shuffle_walking",
    "running",
    "misc_movements"
]

ALL_ACTIVITIES_COMBINED = [
    "sitting_or_standing",
    "lying_down_back",
    "lying_down_stomach",
    "lying_down_right",
    "lying_down_left",
    "walking",
    "ascending_stairs",
    "descending_stairs",
    "shuffle_walking",
    "running",
    "misc_movements"
]


DATA_DIRECTORY = "./all_respeck"
LABEL_TO_INDEX = {label: idx for idx, label in enumerate(ALL_ACTIVITIES_COMBINED)}



In [9]:
def generate_training_data(directory, sequence_length, overlap, file_names, gyro = GYRO): # if gyro is false, only accelerometer data is used

    tagged_data = []

    # group each csv file into their respective areas
    csv_dictionary = file_tagger.tag_directory(directory)

    # iterates through each activity
    for key in csv_dictionary.keys():
        
        activity = key.split("&")[0]
        if activity in ALL_ACTIVITIES:

            # iterates through each csv file for the activity 
            for csv_file in csv_dictionary[key]:
                if csv_file in file_names:
                    if gyro:
                        sequences = sequence_genrator.generate_sequences_from_file_with_gyroscope(directory + "/" + csv_file, sequence_length, overlap, normalise=NORMALISE)
                    else:
                        sequences = sequence_genrator.generate_sequences_from_file_without_gyroscope(directory + "/" + csv_file, sequence_length, overlap, normalise=NORMALISE)

                    # iterate through each generated sequence
                    for sequence in sequences:
                        if activity == "standing" or activity == "sitting":
                            tagged_data.append(("sitting_or_standing", sequence))
                        else:
                            tagged_data.append((activity, sequence))

    print ("there are " + str(len(tagged_data)) + " tagged sequences in the dataset")
    return tagged_data

In [10]:
def train_model_CNN(input_data, labels_encoded, unique_labels, epochs, batch_size, validation_data):
    if GYRO:
        width = 6
    else:
        width = 3
    # Define the CNN model for your specific input shape
    model = Sequential([
        layers.Conv1D(32, 3, activation='relu', input_shape=(SEQUENCE_LENGTH*25, width)),
        layers.MaxPooling1D(2),
        layers.Conv1D(64, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Conv1D(128, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Dropout(0.5),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(len(unique_labels), activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the CNN model
    if len(validation_data[0]) == 0:
        model.fit(input_data, labels_encoded, epochs=epochs, batch_size=batch_size)
    else:
        model.fit(input_data, labels_encoded, epochs=epochs, batch_size=batch_size, validation_data=validation_data)

    return model

In [11]:
def create_sequence_label_lists(tagged_sequences):
    sequences = [sequence for _, sequence in tagged_sequences]
    labels = [label for label, _ in tagged_sequences]
    sequences = np.array(sequences, dtype=np.float32)
    labels_encoded = [LABEL_TO_INDEX[label] for label in labels]
    labels = np.array(labels_encoded)

    return sequences, labels
    

def create_data_sets(dev_size, test_size):

    training_files, dev_files, test_files = split_by_student.split_data(students_in_dev_set= dev_size, students_in_test_set=test_size)

    tagged_training_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, SEQUENCE_OVERLAP, file_names=training_files)
    tagged_dev_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, SEQUENCE_OVERLAP, file_names=dev_files)
    tagged_test_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, SEQUENCE_OVERLAP, file_names=test_files)

    train_data, train_labels = create_sequence_label_lists(tagged_training_sequences)
    dev_data, dev_labels = create_sequence_label_lists(tagged_dev_sequences)
    test_data, test_labels = create_sequence_label_lists(tagged_test_sequences)

    #print(len(train_data), len(train_labels), len(dev_data), len(dev_labels), len(test_data), len(test_labels))

    return train_data, train_labels, dev_data, dev_labels, test_data, test_labels

In [12]:


train_data, train_labels, dev_data, dev_labels, test_data, test_labels = create_data_sets(dev_size=DEV_SIZE, test_size=TEST_SIZE) #10% dev, 10% test


# train and save model (CHOOSE BETWEEN CNN AND LSTM)
model = train_model_CNN(train_data, train_labels, ALL_ACTIVITIES_COMBINED, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(dev_data, dev_labels)) #batch_size, epochs



# Evaluate the model on the test set
if TEST_SIZE >0:
    test_loss, test_accuracy = model.evaluate(test_data, test_labels)
    print (f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

print("labels by index:")
for label, index in LABEL_TO_INDEX.items():
    print(f"{index}: {label}")

if SAVE_MODEL:
# Save the trained model
    model.save(f"models/demo_models/all_physical_classifier.keras")

Train Set: s70, s63, s98, s51, s40, s22, s35, s12, s77, s34, s52, s75, s27, s3, s80, s50, s66, s67, s44, s74, s11, s55, s7, s54, s15, s5, s93, s30, s33, s102, s92, s56, s100, s13, s45, s48, s72, s87, s79, s16, s91, s96, s43, s18, s46, s38, s97, s86, s9, s57, s65, s39, s8, s17, s1, s88, s42, s23, s61, s32, s29
Dev Set: s83, s84, s71, s36, s82
Test Set: s59, s60, s21, s64, s95
there are 71591 tagged sequences in the dataset
there are 5925 tagged sequences in the dataset
there are 5952 tagged sequences in the dataset
Epoch 1/20
7160/7160 [==============================] - 21s 3ms/step - loss: 0.7966 - accuracy: 0.7007 - val_loss: 0.7550 - val_accuracy: 0.7141
Epoch 2/20
7160/7160 [==============================] - 21s 3ms/step - loss: 0.5546 - accuracy: 0.7971 - val_loss: 0.8638 - val_accuracy: 0.7031
Epoch 3/20
7160/7160 [==============================] - 21s 3ms/step - loss: 0.4869 - accuracy: 0.8215 - val_loss: 0.7186 - val_accuracy: 0.7414
Epoch 4/20
5683/7160 [======================>

KeyboardInterrupt: 